# Supermarket basket association pattern mining

In this question, we perform association pattern mining using the supermarket dataset `supermarket.arff` from the [Weka MOOC](https://www.cs.waikato.ac.nz/ml/weka/courses.html).


1. Load the data file `supermarket.arff` into a pandas data frame

2. Remove the following attributes
  - `department*`
  - `non host support`
  - `total`

3.  Select the Apriori algorithm and perform frequent itemset mining with minsup = 0.2 and minconf = 0.8 and find out:
  - The numbers of frequent 2-itemsets, and 3-itemsets.
  - The best three (2) rules with largest confidence. Examine these rules and describe them in your own words.

4. The supermarket manager wishes to boost the sale of fruit and therefore the manager needs to know other itemsets most likely be purchased with fruit to make promotion decisions.
  - Using the same minimum support and minimum confidence value.
  - List the top three itemsets to report to the supermarket manager.

5. Repeat task 3, but using the FP Growth algorithm instead.  
  - Compare the rules found.
  - Are they consistent?

## 0 Upgrade mlxtend
The default version of `mlxtend` on Google Colaborate is too old for this prac
so we must upgrade it. We want something that is at least version 0.18.
Note that code statements beginning with `!` are not python code, but system calls. If you are running this in a personal jupyterlab you might have to update this module a different way.

In [ ]:
! pip install --upgrade 'mlxtend>=0.18'

In [ ]:
# Check we have the right version
import mlxtend
print(mlxtend.__version__)

If you ran the two cells above inreverse order then you'll have to restart the kernel before you can load the newer version of the `mlxtend` module.

To do this: choose "Runitime" -> "Restart runtime".

In [ ]:
import pandas as pd
from scipy.io import arff
import urllib
import urllib.request
import numpy as np

## 1 Load the data file `supermarket.arff` into a pandas data frame

We did this in a previous prac: download the file into your working directory using `urrlib`, load it using `scipy`, and then convert to a `pandas` data frame. The file on the Weka website has a few problems that we need to work around, so I've provided a cleaned version of the data on [GitHub](https://raw.githubusercontent.com/PaulHancock/COMP5009_pracs/main/data/supermarket.arff).

In [ ]:
data_url = 'https://raw.githubusercontent.com/PaulHancock/COMP5009_pracs/main/data/supermarket.arff'
file_name = 'supermarket.arff'
urllib.request.urlretrieve(data_url, file_name)

In [ ]:
# load the data from arff format
data = arff.loadarff('supermarket.arff')
raw_df = pd.DataFrame(data[0])
# The data table is 1 and 0, but we want it to be boolean (true/false) so we
# need to convert from int -> bool
df = raw_df.astype(?)

In [ ]:
df.describe()

## 2 Remove attributes
Remove the following attributes as they have been deemed to be not-useful:
  - `department*`
  - `non host support`
  - `total`


In [ ]:
cols_to_drop = ['non host support', 'total']
# Instead of hand writing all the names that start with department, use a loop
for col in df.columns:
  if col.startswith(?): # choose all the columns which start with the word 'department'
    cols_to_drop.append(col)
print("The folloiwing columns will be dropped:")
print(cols_to_drop)

In [ ]:
df = df.drop(columns=cols_to_drop)

In [ ]:
# confirm we have dropped the columns by showing a summary, we should have 104 cols left, all with descriptive names.
df.describe()

## 3 Select the Apriori algorithm

Select the Apriori algorithm and perform frequent itemset mining with `minsup = 0.2` and `minconf = 0.8` and find out:

- The numbers of frequent 2-itemsets, and 3-itemsets.
- The best three rules with largest confidence. Examine these rules and describe them in your own words.

The `apriori` algorithm is found in the `mlxtend` package, so we import it along with the `association_rules` function.

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
ap_itemsets = apriori(df,
                      min_support=?,  # choose the (relative) minsup
                      use_colnames=True)

In [ ]:
ap_itemsets

Now that we have our itemsets we want to chose those with `2<=k<=3`.
This isn't explicitly stored within our dataframe so we'll make a new column which is just the value of `len(itemsets)`.

In [ ]:
def find_k(row):
  """Return the number of items in the itemset"""
  return len(row['itemsets'])

# Create a new column which counts the number of items in the itemset
ap_itemsets['k'] = ap_itemsets.apply(?, # Apply the function `find_k`
                                     axis=1) # apply the function to each row

In [ ]:
ap_itemsets

In [ ]:
k2_itemsets = np.sum(ap_itemsets['k'] == ?) # count the number of rows where k=2
k3_itemsets = np.sum(ap_itemsets['k'] == ?) # count the number of rows where k=3
print(f"There are {k2_itemsets} itemsets with k=2")
print(f"There are {k3_itemsets} itemsets with k=3")

In [ ]:
# Now lets see the top 10 itemsets
# try either .head() or .nlargest(10,'support')
ap_itemsets.?

Note that the top 10 itemsets are all 1-itemsets. Is this surprising to you?

We use these itemsets to generate association rules with a minimum confidence of 0.8.

In [ ]:
ap_rules = association_rules(ap_itemsets,
                             metric='confidence',
                             min_threshold=?) # choose the minimum confidence value

In [ ]:
ap_rules.head()

Note that the rules above are not sorted by confidence. We should do that ourselves by using the `sort_values` function.

In [ ]:
ap_rules.sort_values('confidence', ascending=False)

Now describe the first three that you see above in your own words.

## 4 Boost fruit sales
The supermarket manager wishes to boost the sale of fruit and therefore the manager needs to know other itemsets most likely be purchased with fruit to make promotion decisions.
  - Using the same minimum support and minimum confidence value.
  - List the top three itemsets to report to the supermarket manager.

In [ ]:
# choose all the rules wihch have "fruit" (not canned fruit) as the consquent
fruit_rules = ap_rules[ap_rules.consequents == frozenset([?])]

fruit_rules.sort_values(?,  # sort based on 'confidence'
                        ascending=False).head(3) # choose the top 3 only

## 5 FP-Growth
Repeat task 3, but using the FP Growth algorithm instead.  
  - Compare the rules found.
  - Are they consistent?

Import the `fpgrowth` function from our `mlxtend` module

In [ ]:
from mlxtend.frequent_patterns import fpgrowth


In [ ]:
fp_itemsets = fpgrowth(df,
                       min_support=?, # choose the minimum support
                       use_colnames=True)

In [ ]:
fp_rules = association_rules(fp_itemsets,
                             metric='confidence',
                             min_threshold=?) # choose the minimum confidence

There are a lot of rules, lets compare just the first 10 most confident rules.

In [ ]:
# Select the top 10 confident rules from each of our algorithms
fp_top_10 = fp_rules.sort_values('confidence', ascending=False).head(10)
ap_top_10 = ap_rules.sort_values('confidence', ascending=False).head(10)

In [ ]:
print("FP-Growth rules")
fp_top_10

In [ ]:
print("Apriori rules")
ap_top_10

Do the above tables agree?